<a href="https://colab.research.google.com/github/marongkang/MLeveryday/blob/main/MLEveryday7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**ML** **day7** 
>今天的目标是 linear regression





In [5]:
import urllib.request
url="https://raw.githubusercontent.com/marongkang/datasets/main/titanic.csv"
response=urllib.request.urlopen(url)
page=response.read()
f=open('titanic.csv','wb')
f.write(page)
!ls -1

sample_data
titanic.csv


In [33]:
import pandas as pd
df=pd.read_csv('titanic.csv',header=0)
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [8]:
#通过索引获取数据，类似numpy的matrix
df.iloc[0,:]

PassengerId                          1
Survived                             0
Pclass                               3
Name           Braund, Mr. Owen Harris
Sex                               male
Age                                 22
SibSp                                1
Parch                                0
Ticket                       A/5 21171
Fare                              7.25
Cabin                              NaN
Embarked                             S
Name: 0, dtype: object

In [9]:
#同上
df.iloc[0,4]

'male'

In [10]:
df.loc[0]
df.loc[1]

PassengerId                                                    2
Survived                                                       1
Pclass                                                         1
Name           Cumings, Mrs. John Bradley (Florence Briggs Th...
Sex                                                       female
Age                                                           38
SibSp                                                          1
Parch                                                          0
Ticket                                                  PC 17599
Fare                                                     71.2833
Cabin                                                        C85
Embarked                                                       C
Name: 1, dtype: object

In [12]:
#返回具有NaN值的行
#NaN原指Not a Number，非数，此处意为空数据
df[pd.isnull(df).any(axis=1)].head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S


In [15]:
#删除具有NaN值的行
df.dropna()
#重置索引值
df.reset_index()
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [34]:
#删除多列
df=df.drop(["Name","Cabin","Ticket"],axis=1)
df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


In [35]:
#映射特征值
#sex male=1,female=0
df["Sex"]=df["Sex"].map({"male":1,"female":0}).astype(int)
#Embarked s=0,c=1,q=2
df["Embarked"]=df["Embarked"].dropna().map({"S":0,"C":1,"Q":2}).astype(int)
#df[df["Survived"]==1].mean()
df.head()
#df.mean()
#第二次执行会报错，因为df内的数据已经被修改了

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,1,22.0,1,0,7.2500,0.0
1,2,1,1,0,38.0,1,0,71.2833,1.0
2,3,1,3,0,26.0,0,0,7.9250,0.0
3,4,1,1,0,35.0,1,0,53.1000,0.0
4,5,0,3,1,35.0,0,0,8.0500,0.0


In [49]:
#lambda表达式创建新特性
#不使用lambda
def getFamilySize(series):
  familySize=series["SibSp"]+series["Parch"]
  return familySize
#SibSp:number of subling or spouse
#Parch:number of parents and children
df["Family_size"]=df[["SibSp","Parch"]].apply(getFamilySize,axis=1)
df.head(n=10)
#使用lambda
def getFSize(ss,pc):
  familySize=ss+pc
  return familySize
df["Family_size"]=df[["SibSp","Parch"]].apply(lambda x:getFSize(x["SibSp"],x["Parch"]),axis=1)
df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Family_size
0,1,0,3,1,22.0,1,0,7.2500,0.0,1
1,2,1,1,0,38.0,1,0,71.2833,1.0,1
2,3,1,3,0,26.0,0,0,7.9250,0.0,0
3,4,1,1,0,35.0,1,0,53.1000,0.0,1
4,5,0,3,1,35.0,0,0,8.0500,0.0,0


In [51]:
#重新组织标题
df=df[["Pclass","Sex","Age","Fare","Embarked","Family_size"]]
df.head()

,Pclass,Sex,Age,Fare,Embarked,Family_size
0,3,1,22.0,7.2500,0.0,1
1,1,0,38.0,71.2833,1.0,1
2,3,0,26.0,7.9250,0.0,0
3,1,0,35.0,53.1000,0.0,1
4,3,1,35.0,8.0500,0.0,0


In [52]:
#保存
df.to_csv("processed_titanic.csv")

In [53]:
!ls -1

processed_titanic.csv
sample_data
titanic.csv
